In [1]:
import pandas as pd
import pyodbc
# File names and directories
survey_dir = 'J:/Projects/Surveys/HHTravel/Survey2019/Data/PSRC_2019_HTS_Deliverable_022020/Weighted_Data_022020/geocoded/'
hh_file_name = '1_household.csv'
person_file_name = '2_person.csv'
vehicle_file_name = '3_vehicle.csv'
day_file_name = '4_day.csv'
trip_file_name = '5_trip.csv'

sql_conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=AWS-PROD-SQL\COHO;DATABASE=Elmer;trusted_connection=true')
parcels_name = "LUV.ParcelDims"
parcels = pd.read_sql('SELECT * FROM '+parcels_name, con = sql_conn)

hh_weight_2017 = 'hh_wt_revised'
hh_weight_2019 = 'hh_wt_2019'
hh_weight_combined = 'hh_wt_combined'
hh_weight_names =[hh_weight_2017, hh_weight_2019, hh_weight_combined]

day_weight_2017 = 'hh_day_wt_revised'
day_weight_2019 = 'hh_day_wt_2019'
day_weight_combined = 'hh_day_wt_combined'
day_weight_names =[day_weight_2017, day_weight_2019, day_weight_combined]

trip_weight_2017 = 'trip_weight_revised'
trip_weight_2019 ='trip_wt_2019'
trip_weight_combined = 'trip_wt_combined'
trip_weight_names =[trip_weight_2017, trip_weight_2019, trip_weight_combined]

codebook_file = 'Combined_Codebook_022020.xlsx'
output_file_hh = 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/hh_summary.xlsx'
output_file_person= 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/person_summary.xlsx'
output_file_day= 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/day_summary.xlsx'
output_file_trip= 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/trip_summary.xlsx'

mode_simple_file =  'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/mode_simple.xlsx'
purpose_simple_file =  'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/purpose_simple.xlsx'

In [2]:
def merge_hh_person_trip(hh,person,trip):
    hh_person =pd.merge(hh, person, on= 'hhid', suffixes=['', 'person'], how ='right')
    hh_person_trip = pd.merge(hh_person, trip,on='personid', suffixes=['','trip'], how ='right')
    return hh_person_trip

def merge_hh_person(hh, person):
    hh_person =pd.merge(hh, person, on= 'hhid', suffixes=['', 'person'], how ='right')
    return hh_person


In [3]:
hh = pd.read_csv(survey_dir+hh_file_name, encoding = 'latin-1')
person= pd.read_csv(survey_dir+person_file_name, encoding = 'latin-1')
#trip = pd.read_csv(survey_dir+trip_file_name, encoding = 'latin-1')
#day= pd.read_csv(survey_dir+day_file_name, encoding='latin-1')

C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (14,46,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8,44,45,53,56,57,65,72,73,81) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
codebook_values = pd.read_excel(survey_dir+codebook_file, sheetname='Values')
codebook_variables =pd.read_excel(survey_dir+codebook_file, sheet_name='Data Overview')

C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [5]:
mode_simple= pd.read_excel(mode_simple_file)
purpose_simple = pd.read_excel(purpose_simple_file)

In [6]:
person_detail = merge_hh_person(hh, person)

In [7]:
#person_trip = merge_hh_person_trip(hh,person,trip)

In [8]:
def lookup_codes(grouped_total, item):
    values=codebook_values.loc[codebook_values['variable']==item]
    variable =codebook_variables.loc[codebook_variables['variable']==item, 'description'].values[0]
    missing_values_df = pd.DataFrame({'variable': [item, item, item],'value':[-9999,-9998, 995], 'label':
                                         ['Missing:Technical Error', 'Missing: Non-response', 'Missing:Skip logic']})
    values = values.append(missing_values_df)
    summary=pd.merge(values, grouped_total,left_on='value', right_on=item)
    summary[item+'_variable']=variable
    return summary

In [9]:
person_commute= person_detail.groupby('commute_mode').sum()[hh_weight_2019].reset_index()

person_commute =lookup_codes(person_commute, 'commute_mode')

In [10]:
person_commute

,variable,value,label,commute_mode,hh_wt_2019,commute_mode_variable
0,commute_mode,1,Drive alone,1.0,1.368811e+06,Usual way of commuting to current location/office
1,commute_mode,2,Carpool ONLY with other household members,2.0,1.615599e+05,Usual way of commuting to current location/office
2,commute_mode,3,Carpool with other people not in household (ma...,3.0,2.387081e+04,Usual way of commuting to current location/office
3,commute_mode,4,Motorcycle/moped/scooter,4.0,0.000000e+00,Usual way of commuting to current location/office
4,commute_mode,5,Vanpool,5.0,2.375706e+04,Usual way of commuting to current location/office
5,commute_mode,6,Bicycle or e-bike,6.0,6.492544e+04,Usual way of commuting to current location/office
6,commute_mode,7,"Walk, jog, or wheelchair",7.0,3.286490e+04,Usual way of commuting to current location/office
7,commute_mode,8,Bus (public transit),8.0,1.646594e+05,Usual way of commuting to current location/office
8,commute_mode,9,Private bus or shuttle,9.0,1.151994e+04,Usual way of commuting to current location/office
9,commute_mode,10,Paratransit,10.0,0.000000e+00,Usual way of commuting to current location/office


In [11]:
person_commute_county = person_detail.groupby(['commute_mode', 'sample_county']).sum()[hh_weight_2019].reset_index()

person_modes_cnty =lookup_codes(person_commute_county, 'commute_mode')

In [12]:
person_modes_cnty.to_clipboard()

In [13]:
person_modes_puma = person_detail.groupby(['commute_mode', 'final_home_puma10']).sum()[hh_weight_2019].reset_index()
person_modes_puma =lookup_codes(person_modes_puma, 'commute_mode')
person_modes_puma.to_clipboard()

In [14]:
pd.DataFrame(person_detail.columns).to_clipboard()

In [15]:
person_detail = person_detail.dropna(subset=['final_home_parcel'])
person_detail['home_parcel_int'] = person_detail['final_home_parcel'].astype('int32')
parcels['ParcelIDInt'] = parcels['ParcelID'].astype('int32')
people_parcels=pd.merge(person_detail, parcels, left_on='home_parcel_int', right_on='ParcelIDInt', )

person_mode_city=people_parcels.groupby(['commute_mode', 'CityName']).sum()[hh_weight_2019].reset_index()
person_mode_city =lookup_codes(person_mode_city, 'commute_mode')
person_mode_city.to_clipboard()

In [16]:
person_commute_17= person_detail.groupby('commute_mode').sum()[hh_weight_2017].reset_index()

person_commute_17 =lookup_codes(person_commute_17, 'commute_mode')

In [17]:
person_commute_17

,variable,value,label,commute_mode,hh_wt_revised,commute_mode_variable
0,commute_mode,1,Drive alone,1.0,1.263669e+06,Usual way of commuting to current location/office
1,commute_mode,2,Carpool ONLY with other household members,2.0,9.246115e+04,Usual way of commuting to current location/office
2,commute_mode,3,Carpool with other people not in household (ma...,3.0,5.113352e+04,Usual way of commuting to current location/office
3,commute_mode,4,Motorcycle/moped/scooter,4.0,1.712355e+04,Usual way of commuting to current location/office
4,commute_mode,5,Vanpool,5.0,4.534550e+04,Usual way of commuting to current location/office
5,commute_mode,6,Bicycle or e-bike,6.0,5.873954e+04,Usual way of commuting to current location/office
6,commute_mode,7,"Walk, jog, or wheelchair",7.0,4.244695e+04,Usual way of commuting to current location/office
7,commute_mode,8,Bus (public transit),8.0,2.486367e+05,Usual way of commuting to current location/office
8,commute_mode,9,Private bus or shuttle,9.0,4.508655e+03,Usual way of commuting to current location/office
9,commute_mode,10,Paratransit,10.0,2.926922e+02,Usual way of commuting to current location/office


In [18]:
person_commute_county_17 = person_detail.groupby(['commute_mode', 'sample_county']).sum()[hh_weight_2017].reset_index()

person_modes_cnty =lookup_codes(person_commute_county_17, 'commute_mode')

In [19]:
person_modes_cnty.to_clipboard()

In [20]:
parcels.head()

,ParcelDimID,ParcelID,BaseYear,CityName,CityRGExisting,CityRGProposed,CountyName,Census2010Block,Census2010BlockGroup,Census2010Tract,GrowthCenterName,TAZID,ZoneAreaType,FAZID,FAZLargeAreaName,XCoordStatePlane,YCoordStatePlane,Geom,ParcelIDInt
0,1,729205,2014,Black Diamond,Small,CitiesTowns,King,530330316032028,530330316032,53033031603,,2046,Core & Larger Suburban Cities,3310,SE King,1350280.0,116175.0,b'\xed\x08\x00\x00\x01\x0c\x00\x00\x00\x00\x88...,729205
1,2,735566,2014,King-Rural,Rural,Rural,King,530330326011054,530330326011,53033032601,,1950,Core & Larger Suburban Cities,6505,King Other,1376310.0,206838.0,b'\xed\x08\x00\x00\x01\x0c\x00\x00\x00\x006\x0...,735566
2,3,735828,2014,Seattle,Metro,Metro,King,530330098003019,530330098003,53033009800,,698,Core & Larger Suburban Cities,5720,Seattle,1258830.0,210957.0,b'\xed\x08\x00\x00\x01\x0c\x00\x00\x00\x00N53A...,735828
3,4,738279,2014,SeaTac,Core,Core,King,530330284032007,530330284032,53033028403,,1011,Core & Larger Suburban Cities,3705,SW King,1280700.0,162308.0,b'\xed\x08\x00\x00\x01\x0c\x00\x00\x00\x00\xbc...,738279
4,5,742042,2014,Seattle,Metro,Metro,King,530330117004006,530330117004,53033011700,,847,Core & Larger Suburban Cities,5915,Seattle,1284520.0,191150.0,b'\xed\x08\x00\x00\x01\x0c\x00\x00\x00\x00\xa8...,742042


In [21]:
person_modes_puma_17 = person_detail.groupby(['commute_mode', 'final_home_puma10']).sum()[hh_weight_2017].reset_index()
person_modes_puma_17 =lookup_codes(person_modes_puma_17, 'commute_mode')
person_modes_puma_17.to_clipboard()

In [22]:
person_detail_commute_mode = person_detail.dropna(subset=['commute_mode'])

In [23]:
person_detail_commute_mode.count()

commute_mode
1.0      3167
2.0       336
3.0       132
4.0        12
5.0        54
6.0       321
7.0       647
8.0      1598
9.0        54
10.0        1
11.0       48
12.0      224
13.0        8
14.0       32
15.0        1
16.0       34
17.0       14
18.0        3
19.0       12
97.0        2
995.0    2471
Name: hhid, dtype: int64

In [24]:
person_detail_17_commute_mode =person_detail_commute_mode.loc[person_detail_commute_mode['survey_year']==2017]

In [25]:
person_detail_17_commute_mode.groupby(['hhid']).nunique()['hhid'].count()

2513

In [46]:
person_count=person_detail_17_commute_mode.groupby(['hhid']).first()['final_cnty'].reset_index()

county_count=person_count.groupby('final_cnty').count()['hhid'].reset_index()

In [48]:
lookup_codes(county_count, 'final_cnty').to_clipboard()

In [52]:
person_detail_17_commute_mode['final_home_puma10_int']=person_detail_17_commute_mode['final_home_puma10'].astype('int32')

C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
seattle_commute=person_detail_17_commute_mode.loc[((person_detail_17_commute_mode['final_home_puma10_int']>=11601) & (person_detail_17_commute_mode['final_home_puma10_int']<=11605))]

In [56]:
seattle_commute.groupby(['hhid']).nunique()['hhid'].count()

1529